In [2]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score, roc_auc_score, make_scorer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix, plot_confusion_matrix

In [3]:
filename = "../../data/iPinYou-all/hdf/train_input_part_0.h5"
with pd.HDFStore(filename, mode='r') as hdf_y:
    input_data = pd.read_hdf(hdf_y)

filename = "../../data/iPinYou-all/hdf/train_output_part_0.h5"
with pd.HDFStore(filename, mode='r') as hdf_y:
    tgt_data = pd.read_hdf(hdf_y)
    pos_neg = np.sum(tgt_data['click'] == 1)
    num_neg = np.sum(tgt_data['click'] == 0)
    print ("Pos: %d, Neg: %d" % (pos_neg, num_neg))

final_data = pd.concat([input_data, tgt_data], axis=1, join='inner')
input_columns=['weekday', 'hour', 'IP', 'region', 'city', 'adexchange', 'domain',
   'slotid', 'slotwidth', 'slotheight', 'slotvisibility', 'slotformat',
   'creative', 'advertiser', 'useragent', 'slotprice']
sparse_features = ['weekday', 'hour', 'region', 'city', 'adexchange', 'slotwidth', 'slotheight', \
        'slotvisibility', 'slotformat', 'creative', 'advertiser', 'useragent', 'slotprice', \
        'IP', 'domain', 'slotid']
dense_features = []
target= 'click'

Pos: 1281, Neg: 1998718


In [4]:
final_data.head()

,weekday,hour,IP,region,city,adexchange,domain,slotid,slotwidth,slotheight,slotvisibility,slotformat,creative,advertiser,useragent,slotprice,click
0,5,9,129214,705001,705056,705406,745192,936648,937446,937463,937465,937478,937525,937621,937655,937665,0
1,5,9,533685,705018,705257,705407,737280,873692,937438,937457,937467,937477,937521,937619,937657,937666,0
2,5,9,619037,705033,705400,705406,745683,935566,937446,937463,937465,937478,937597,937619,937655,937665,0
3,5,9,264384,705033,705400,705406,751770,935528,937436,937456,937467,937478,937532,937620,937655,937665,0
4,5,9,633959,705010,705162,705406,750179,937217,937436,937456,937465,937479,937598,937619,937628,937665,0


In [5]:
X = final_data.drop(target, axis=1).copy()
y = final_data[target].copy()

In [6]:
y.unique()

array([0, 1])

In [7]:
# Do one hot encoding if you want
# Use sklearn or pandas one hot encoding
X_encoded = X

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=2020, stratify=y)
clf_xgb = xgb.XGBClassifier(objective='binary:logistic',
                            gamma=0.25,
                            learn_rate=0.1,
                            max_depth=5,
                            reg_lambda=0.5,
                            scale_pos_weight=3,
                            subsample=0.9,
                            n_estimators=100,
#                             use_label_encoder=False,
                            seed=2020)
clf_xgb.fit(X_train,
            y_train,
            verbose=True,
            early_stopping_rounds=10,
            eval_metric='auc',     # used logloss as well
            eval_set=[(X_test, y_test)])

bst = clf_xgb.get_booster()

C:\Users\dipti\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[12:57:56] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:576: 
Parameters: { "learn_rate" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[0]	validation_0-auc:0.58890
[1]	validation_0-auc:0.60226
[2]	validation_0-auc:0.65912
[3]	validation_0-auc:0.68408
[4]	validation_0-auc:0.68482
[5]	validation_0-auc:0.70238
[6]	validation_0-auc:0.70294
[7]	validation_0-auc:0.70782
[8]	validation_0-auc:0.69340
[9]	validation_0-auc:0.69093
[10]	validation_0-auc:0.68828
[11]	validation_0-auc:0.69934
[12]	validation_0-auc:0.69556
[13]	validation_0-auc:0.71080
[14]	validation_0-auc:0.72311
[15]	validation_0-auc:0.72079
[16]	validation_0-auc:0.72143
[17]	validation_0-auc:0.71853
[18]	validation_0-auc:0.71596
[19]	validation_0-auc

In [9]:
bst